### Attempt 1: Beautiful Soup

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

scraping headlines

In [36]:
# Function to scrape all headlines on WSJ on a given day
def scrape_wsj_date(date):
    # Base URL
    base_url = f'https://www.wsj.com/news/archive/{str(date.year)}/{str(date.month).zfill(2)}/{str(date.day).zfill(2)}?page='

    # Initialize lists to store data
    all_articles = []

    # Create page counter
    page_num = 1

    # Define header
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

    # Loop through each page
    while True:
        # URL of the webpage to scrape
        url = base_url + str(page_num)

        # Open the webpage
        response = requests.get(url, headers=headers)

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all articles on the page
        articles = soup.find_all('article')

        # Exit loop if no more articles on page
        if not articles: 
            break
        
        all_articles.extend(articles)
        
        # Increment page
        page_num += 1

    return all_articles

In [37]:
date = datetime.date(2023, 3, 19)

temp = scrape_wsj_date(date)

In [38]:
len(temp)

58

scraping article body

In [39]:
# Base URL
url = f'https://www.wsj.com/world/middle-east/syria-says-israeli-strikes-near-aleppo-kill-wound-dozens-27a66ace'

# Define header
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

In [41]:
soup

<html><head><title>wsj.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={'rt':'c','cid':'AHrlqAAAAAMAf3Q7YQAmGH0Aa4lleQ==','hsh':'D428D51E28968797BC27FB9153435D','t':'fe','s':47192,'e':'e458cbe70e6c77951cb530148bba13711065fe2b9bdd9b3c37ba60097918b0fd','host':'geo.captcha-delivery.com'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>

### Attempt 2: Selenium

scraping content

In [9]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
import time
import random

In [ ]:
# Path to the Microsoft Edge WebDriver executable
edge_driver_path = r'C:\Users\MarcoHui\Desktop\msedgedriver.exe'  # Update this with the actual path

# Initialize Edge WebDriver with the specified path
edge_service = Service(edge_driver_path)
edge_service.start()
driver = webdriver.Edge(service=edge_service)

# URL of the article to scrape
url = 'https://www.wsj.com/world/middle-east/-jose-andres-chef-gaza-israel-war-ea9512f8'

# Open the URL in the browser
driver.get(url)

# Introduce a random delay between 3 to 5 seconds to mimic human-like behavior
time.sleep(random.uniform(3, 5))

# Find the article body element
article_body_element = driver.find_element(By.CLASS_NAME, 'article-body')

# Extract text from the article body
article_body = article_body_element.text

# Print or process the article body as needed
print(article_body)

# Close the browser
driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

# URL of the article
url = 'https://www.wsj.com/world/middle-east/syria-says-israeli-strikes-near-aleppo-kill-wound-dozens-27a66ace'

# Set up the Selenium driver with Edge
driver = webdriver.Edge()  # Use Edge WebDriver

# Load the page
driver.get(url)

# Wait for the sign-in link to appear and click it
sign_in_link = driver.find_element_by_link_text("Sign In")
sign_in_link.click()

# Wait for the sign-in page to load
time.sleep(2)

# Enter your username and password
username_field = driver.find_element_by_id("username")
username_field.send_keys("")  # Replace "your_username" with your actual username
password_field = driver.find_element_by_id("password")
password_field.send_keys("abc123")  # Replace "your_password" with your actual password

# Submit the login form
password_field.send_keys(Keys.RETURN)

# Wait for the page to load after login
time.sleep(5)

# Now you can extract the page source
page_source = driver.page_source

# Close the driver
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Now you can scrape the content as you normally would with BeautifulSoup
